# ProtoCLR

https://huggingface.co/ilyassmoummad/ProtoCLR

In [1]:
!git clone https://huggingface.co/ilyassmoummad/ProtoCLR

Cloning into 'ProtoCLR'...
remote: Enumerating objects: 93, done.
remote: Counting objects: 100% (90/90), done.
remote: Compressing objects: 100% (89/89), done.
remote: Total 93 (delta 45), reused 0 (delta 0), pack-reused 3 (from 1)
Unpacking objects: 100% (93/93), 41.14 KiB | 957.00 KiB/s, done.
Filtering content: 100% (4/4), 315.15 MiB | 27.91 MiB/s, done.


In [19]:
!pip install -r /content/ProtoCLR/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 kB 3.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 71.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 63.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.6/755.6 MB 722.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 67.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.3 M

In [20]:
!pip install pydub

In [21]:
# Load file
from google.colab import files
uploaded = files.upload()

Saving XC267683.wav to XC267683.wav


In [22]:
# Play XC267683.wav
import IPython
IPython.display.Audio('XC267683.wav')

In [14]:
# from pydub import AudioSegment

# # Input and output file paths
# input_file = "/content/XC267683.ogg"
# output_file = "/content/XC267683.wav"

# # Load the OGG file
# audio = AudioSegment.from_file(input_file, format="ogg")

# # Export as WAV
# audio.export(output_file, format="wav")
# print(f"Converted {input_file} to {output_file}")

Converted /content/XC267683.ogg to /content/XC267683.wav


In [24]:
# cd to ProtoCLR
%cd /content/ProtoCLR

/content/ProtoCLR


In [25]:
# get cwd
import os
os.getcwd()

'/content/ProtoCLR'

In [28]:
import torch
from cvt import cvt13  # Import model architecture
from melspectrogram import MelSpectrogramProcessor  # Import Mel spectrogram processor
import torchaudio
from torchaudio.transforms import Resample

# Initialize the preprocessor and model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
preprocessor = MelSpectrogramProcessor(device=device)
model = cvt13().to(device)

# Load weights trained using Cross-Entropy
model.load_state_dict(torch.load("ce.pth", map_location="cpu")['encoder'])

# Load weights trained using SimCLR (self-supervised contrastive learning)
model.load_state_dict(torch.load("simclr.pth", map_location="cpu"))

# Load weights trained using SupCon (supervised contrastive learning)
model.load_state_dict(torch.load("supcon.pth", map_location="cpu"))

# Load weights trained using ProtoCLR (supervised contrastive learning using prototypes)
model.load_state_dict(torch.load("protoclr.pth", map_location="cpu"))

# Optional: Move the model to GPU for faster processing if available using : model = model.to('cuda') , for instance.
model.eval()

# Load and preprocess a sample audio waveform
def load_waveform(file_path, target_sample_rate=16000):
    # Replace with audio loading code, e.g., torchaudio to load and resample
    # Fill here
    waveform, sample_rate = torchaudio.load(file_path)

    # Resample if necessary
    if sample_rate != target_sample_rate:
        resampler = Resample(orig_freq=sample_rate, new_freq=target_sample_rate)
        waveform = resampler(waveform)

    return waveform

waveform = load_waveform("/content/XC267683.wav").to(device)  # Load your audio file here and convert it to a PyTorch tensor.

# Ensure waveform is sampled at 16 kHz, then pad/chunk as needed for 6s length
input_tensor = preprocessor.process(waveform).unsqueeze(0)  # Add batch dimension

# Run the model on the preprocessed audio
with torch.no_grad():
    output = model(input_tensor)
    print(output)
    print("Model output shape:", output.shape)

tensor([-2.4457e-01,  1.8223e-01,  2.0181e-01,  6.0115e-01,  4.2635e-01,
         6.1178e-01,  1.4896e-01,  1.5647e+00, -6.4761e-03, -3.1457e-01,
        -8.9587e-02, -1.5217e+00,  7.3281e-01,  5.2613e-01,  8.6977e-01,
         1.2215e+00, -3.6874e-01,  2.1697e-02, -7.3876e-01,  7.7171e-03,
         1.5957e-01,  6.0275e-01, -7.1975e-01,  8.7835e-01, -3.1099e-01,
        -7.2741e-02, -8.7675e-02, -1.4797e-01, -5.1775e-01,  5.0220e-01,
        -6.8661e-01, -4.9606e-01, -1.2686e+00,  1.1504e-01,  5.9826e-01,
        -1.4766e+00, -1.0293e+00, -4.3403e-01, -3.5867e-01, -1.0099e+00,
         6.9272e-01,  8.7932e-01,  1.4265e+00,  2.0261e-01, -5.4250e-01,
         4.0648e-01,  6.3792e-01,  1.0620e+00,  6.4828e-01, -1.6143e+00,
        -1.5885e-01, -1.2590e+00,  4.9019e-01, -5.7127e-01,  1.7902e-01,
        -3.1925e-02, -2.0535e-01, -2.0496e-01,  1.5759e-01,  2.3452e-01,
        -6.1840e-01,  5.3674e-01, -1.4958e+00, -6.2212e-01,  3.7179e-01,
        -8.2392e-01,  7.3067e-01,  6.3828e-01,  8.7